In [60]:
import pandas as pd
import numpy as np
import os

results_dir = "Analyzer Results"

recording_order = [15, 2, 1, 6, 10, 4]# interpulse intervals

IPI = {

    "ASR_control":      None,
    "gap_depth":        50,
    "gap_duration_10":  10,
    "gap_duration_20":  20,
    "gap_duration_4":   4,
    "gap_duration_50":  50,
    "gap_duration_8":   8,
    "offset_ASR":       None,
    "offset_PPI_10":    None,
    "offset_PPI_12":    None,
    "offset_PPI_14":    None,
    "offset_PPI_16":    None,
    "offset_PPI_18":    None,
    "offset_PPI_20":    None,
    "offset_PPI_4":     None,
    "offset_PPI_6":     None,
    "offset_PPI_8":     None,
    "offset_PPI_50":    None,
    "tone_in_noise":    50

}
prepulse_intensity = {

    "ASR_control":      120,
    "gap_depth":        30,
    "gap_duration_10":  60,
    "gap_duration_20":  60,
    "gap_duration_4":   60,
    "gap_duration_50":  60,
    "gap_duration_8":   60,
    "offset_ASR":       0,
    "offset_PPI_10":    0,
    "offset_PPI_12":    0,
    "offset_PPI_14":    0,
    "offset_PPI_16":    0,
    "offset_PPI_18":    0,
    "offset_PPI_20":    0,
    "offset_PPI_4":     0,
    "offset_PPI_6":     0,
    "offset_PPI_8":     0,
    "offset_PPI_50":    0,
    "tone_in_noise":    80
}
# ...existing code...

dfs = {}
dfs_fit = {}

files = [file for file in os.listdir(results_dir) if file.endswith(('.xlsx', '.xls'))]

for file in files:
    file_path = os.path.join(results_dir, file)
    key = file.split('.')[0]
    if key.endswith('_FIT'):
        dfs_fit[key] = pd.read_excel(file_path)
    else:
        dfs[key] = pd.read_excel(file_path)

print("Raw dataframes:", list(dfs.keys()))
print("Fit dataframes:", list(dfs_fit.keys()))
print(f"Found {len(dfs)} raw and {len(dfs_fit)} fit dataframes in {results_dir}.")
if len(dfs) != len(dfs_fit):
    print("Warning: Mismatch between raw and fit dataframes!")

experiments = dfs[list(dfs.keys())[1]]['experiment'].unique().tolist()
variables = dfs[list(dfs.keys())[1]].columns[4:].tolist()
parameters = dfs[list(dfs.keys())[1]].columns[:4].tolist()
print(f"\nFound {len(experiments)} experiments, {len(variables)} variables and {len(parameters)} parameters:")
print(" "+', '.join(experiments))
print(" "+', '.join(variables))
print(" "+', '.join(parameters))

Raw dataframes: ['RESULTS', 'RESULTS_MERGED', 'RESULTS_MERGED_DATE', 'RESULTS_MERGED_EXP', 'RESULTS_MTT', 'RESULTS_MTT_MERGED', 'RESULTS_MTT_MERGED_DATE', 'RESULTS_MTT_MERGED_EXP', 'RESULTS_TT', 'RESULTS_TT_MERGED', 'RESULTS_TT_MERGED_DATE', 'RESULTS_TT_MERGED_EXP', 'SegmentOrder']
Fit dataframes: ['RESULTS_FIT', 'RESULTS_MERGED_DATE_FIT', 'RESULTS_MERGED_EXP_FIT', 'RESULTS_MERGED_FIT', 'RESULTS_MTT_FIT', 'RESULTS_MTT_MERGED_DATE_FIT', 'RESULTS_MTT_MERGED_EXP_FIT', 'RESULTS_MTT_MERGED_FIT', 'RESULTS_TT_FIT', 'RESULTS_TT_MERGED_DATE_FIT', 'RESULTS_TT_MERGED_EXP_FIT', 'RESULTS_TT_MERGED_FIT', 'SegmentOrder_FIT']
Found 13 raw and 13 fit dataframes in Analyzer Results.

Found 18 experiments, 9 variables and 4 parameters:
 ASR_control, gap_depth, tone_in_noise, gap_duration_4, gap_duration_8, gap_duration_10, gap_duration_20, gap_duration_50, offset_PPI_4, offset_PPI_6, offset_PPI_8, offset_PPI_10, offset_PPI_12, offset_PPI_14, offset_PPI_16, offset_PPI_18, offset_PPI_20, offset_PPI_50
 rea

---
Expectations:
- date fluctuations
- no repetition differences
- sex differences in strength metrics
- experiment differences, in particular with increased gap / offset
---

In [61]:
# get S from ASR Control
# get PPS from PPI experiments
# get PPSoffset from offset experiments

#calculate %PPI and %PPIoffset

#split all by date and sex

In [62]:
# --- ASR_control ---
asr_data_ctrl = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        asr_control = df[df['experiment'] == 'ASR_control'].copy()
        asr_control['peakValue'] = pd.to_numeric(asr_control['peakValue'], errors='coerce')
        asr_data_ctrl.append(asr_control[['date', 'sex', 'peakValue']])

ctrl_dict = {}
if asr_data_ctrl:
    asr_df = pd.concat(asr_data_ctrl, ignore_index=True)
    avg_by_date_sex = asr_df.groupby(['date', 'sex'])['peakValue'].mean().reset_index()
    print("Average peakValue from ASR_control experiments split by date and sex:")
    #print(avg_by_date_sex)
    overall_avg = asr_df['peakValue'].mean()
    print(f"Overall average peakValue from ASR_control experiments: {overall_avg:.2f}")
    ctrl_dict = {(row['date'], row['sex']): row['peakValue'] for _, row in avg_by_date_sex.iterrows()}
else:
    print("No ASR_control experiments with date, sex, and peakValue found.")


# --- tone_in_noise ---
asr_ttn = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        ttn_control = df[df['experiment'] == 'tone_in_noise'].copy()
        ttn_control['peakValue'] = pd.to_numeric(ttn_control['peakValue'], errors='coerce')
        asr_ttn.append(ttn_control[['date', 'sex', 'peakValue']])

ttn_dict = {}
if asr_ttn:
    ttn_df = pd.concat(asr_ttn, ignore_index=True)
    avg_by_date_sex = ttn_df.groupby(['date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from tone_in_noise experiments split by date and sex:")
    #print(avg_by_date_sex)
    overall_avg = ttn_df['peakValue'].mean()
    print(f"Overall average peakValue from tone_in_noise experiments: {overall_avg:.2f}")
    ttn_dict = {(row['date'], row['sex']): row['peakValue'] for _, row in avg_by_date_sex.iterrows()}
else:
    print("\nNo tone_in_noise experiments with date, sex, and peakValue found.")


# --- Average peakValue across selected experiments (excluding ASR_control, tone_in_noise, offset_ASR, gap_depth) ---
excluded_experiments = {'ASR_control', 'tone_in_noise', 'offset_ASR', 'gap_depth'}
multi_exp_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        mask = ~df['experiment'].isin(excluded_experiments)
        selected = df[mask].copy()
        selected['peakValue'] = pd.to_numeric(selected['peakValue'], errors='coerce')
        multi_exp_data.append(selected[['experiment', 'date', 'sex', 'peakValue']])

mix_dict = {}
if multi_exp_data:
    multi_exp_df = pd.concat(multi_exp_data, ignore_index=True)
    avg_by_exp_date_sex = multi_exp_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from selected experiments split by experiment, date, and sex:")
    #print(avg_by_exp_date_sex)
    overall_avg = multi_exp_df['peakValue'].mean()
    print(f"Overall average peakValue from selected experiments: {overall_avg:.2f}")
    mix_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_exp_date_sex.iterrows()}
else:
    print("\nNo selected experiments with date, sex, and peakValue found.")


# --- Average peakValue for gap_duration experiments ---
gap_duration_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        gap_mask = df['experiment'].str.startswith('gap_duration')
        gap_df = df[gap_mask].copy()
        gap_df['peakValue'] = pd.to_numeric(gap_df['peakValue'], errors='coerce')
        gap_duration_data.append(gap_df[['experiment', 'date', 'sex', 'peakValue']])

gap_dict = {}
if gap_duration_data:
    gap_duration_df = pd.concat(gap_duration_data, ignore_index=True)
    avg_by_gap_date_sex = gap_duration_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from gap_duration experiments split by experiment, date, and sex:")
    #print(avg_by_gap_date_sex)
    overall_avg = gap_duration_df['peakValue'].mean()
    print(f"Overall average peakValue from gap_duration experiments: {overall_avg:.2f}")
    gap_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_gap_date_sex.iterrows()}
else:
    print("\nNo gap_duration experiments with date, sex, and peakValue found.")


# --- Average peakValue for offset_PPI experiments ---
offset_ppi_data = []

for df in dfs.values():
    if {'experiment', 'peakValue', 'date', 'sex'}.issubset(df.columns):
        offset_mask = df['experiment'].str.startswith('offset_PPI')
        offset_df = df[offset_mask].copy()
        offset_df['peakValue'] = pd.to_numeric(offset_df['peakValue'], errors='coerce')
        offset_ppi_data.append(offset_df[['experiment', 'date', 'sex', 'peakValue']])

offset_dict = {}
if offset_ppi_data:
    offset_ppi_df = pd.concat(offset_ppi_data, ignore_index=True)
    avg_by_offset_date_sex = offset_ppi_df.groupby(['experiment', 'date', 'sex'])['peakValue'].mean().reset_index()
    print("\nAverage peakValue from offset_PPI experiments split by experiment, date, and sex:")
    #print(avg_by_offset_date_sex)
    overall_avg = offset_ppi_df['peakValue'].mean()
    print(f"Overall average peakValue from offset_PPI experiments: {overall_avg:.2f}")
    offset_dict = {(row['experiment'], row['date'], row['sex']): row['peakValue'] for _, row in avg_by_offset_date_sex.iterrows()}
else:
    print("\nNo offset_PPI experiments with date, sex, and peakValue found.")



print()
print(ctrl_dict)
print(ttn_dict)
print(mix_dict)
print(gap_dict)
print(offset_dict)


Average peakValue from ASR_control experiments split by date and sex:
Overall average peakValue from ASR_control experiments: 141.26

Average peakValue from tone_in_noise experiments split by date and sex:
Overall average peakValue from tone_in_noise experiments: 103.85

Average peakValue from selected experiments split by experiment, date, and sex:
Overall average peakValue from selected experiments: 126.38

Average peakValue from gap_duration experiments split by experiment, date, and sex:
Overall average peakValue from gap_duration experiments: 135.56

Average peakValue from offset_PPI experiments split by experiment, date, and sex:
Overall average peakValue from offset_PPI experiments: 121.88

{('April16', 'female'): 77.125, ('April16', 'male'): 136.3351851851852, ('June26', 'female'): 123.66666666666667, ('June26', 'male'): 218.2888888888889, ('May20', 'female'): 98.80625, ('May20', 'male'): 186.5375}
{('April16', 'female'): 52.55208333333333, ('April16', 'male'): 110.676190476190

In [63]:
from scipy.stats import ttest_ind

# --- Calculate overall %PPI for each experiment type ---
def calc_overall_percent_ppi(ctrl_dict, exp_dict, label):
    ppi_values = []
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi = 100 * (S - PPS) / S
            ppi_values.append(ppi)
    if ppi_values:
        overall_ppi = np.nanmean(ppi_values)
        print(f"\nOverall %PPI for {label}: {overall_ppi:.2f}")
    else:
        print(f"\nNo valid %PPI values for {label}.")
    return ppi_values

# Usage for each experiment type
ppi_ttn_vals = calc_overall_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")
ppi_mix_vals = calc_overall_percent_ppi(ctrl_dict, mix_dict, "selected experiments")
ppi_gap_vals = calc_overall_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_offset_vals = calc_overall_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")

# --- Statistical significance between groups ---
print("\nSignificance tests (t-test, unequal variance):")
groups = {
    "tone_in_noise": ppi_ttn_vals,
    "selected experiments": ppi_mix_vals,
    "gap_duration": ppi_gap_vals,
    "offset_PPI": ppi_offset_vals
}

labels = list(groups.keys())
for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        group1 = groups[labels[i]]
        group2 = groups[labels[j]]
        # Only test if both groups have at least 2 values
        if len(group1) > 1 and len(group2) > 1:
            stat, pval = ttest_ind(group1, group2, nan_policy='omit', equal_var=False)
            print(f"{labels[i]} vs {labels[j]}: p = {pval:.4f}")
        else:
            print(f"{labels[i]} vs {labels[j]}: Not enough data for t-test.")


Overall %PPI for tone_in_noise: 27.20

Overall %PPI for selected experiments: 9.60

Overall %PPI for gap_duration: 3.10

Overall %PPI for offset_PPI: 12.85

Significance tests (t-test, unequal variance):
tone_in_noise vs selected experiments: p = 0.0048
tone_in_noise vs gap_duration: p = 0.0007
tone_in_noise vs offset_PPI: p = 0.0117
selected experiments vs gap_duration: p = 0.0013
selected experiments vs offset_PPI: p = 0.0227
gap_duration vs offset_PPI: p = 0.0000


In [64]:
def calc_percent_ppi(ctrl_dict, exp_dict, label):
    # Average %PPI per (experiment, date, sex) or (date, sex)
    percent_ppi = {}
    values_by_group = {}
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            # (date, sex)
            group_key = key
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            # (experiment, date, sex)
            group_key = key
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi_val = 100 * (S - PPS) / S
        else:
            ppi_val = np.nan
        if group_key not in values_by_group:
            values_by_group[group_key] = []
        values_by_group[group_key].append(ppi_val)
    # Average over all values for each group
    for group_key, vlist in values_by_group.items():
        vlist = [v for v in vlist if pd.notna(v)]
        percent_ppi[group_key] = np.nanmean(vlist) if vlist else np.nan
    print(f"\n%PPI for {label} (averaged per experiment, date, sex):")
    for k, v in percent_ppi.items():
        print(f"{k}: {v:.2f}" if pd.notna(v) else f"{k}: NaN")
    return percent_ppi


ppi_offset = calc_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")
ppi_gap = calc_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_ttn = calc_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")


%PPI for offset_PPI (averaged per experiment, date, sex):
('offset_PPI_10', 'April16', 'female'): 23.12
('offset_PPI_10', 'April16', 'male'): 16.29
('offset_PPI_10', 'June26', 'female'): 14.84
('offset_PPI_10', 'June26', 'male'): 17.58
('offset_PPI_10', 'May20', 'female'): -1.27
('offset_PPI_10', 'May20', 'male'): 11.62
('offset_PPI_12', 'April16', 'female'): 21.85
('offset_PPI_12', 'April16', 'male'): 12.03
('offset_PPI_12', 'June26', 'female'): 7.61
('offset_PPI_12', 'June26', 'male'): 18.82
('offset_PPI_12', 'May20', 'female'): 8.89
('offset_PPI_12', 'May20', 'male'): 13.37
('offset_PPI_14', 'April16', 'female'): 18.29
('offset_PPI_14', 'April16', 'male'): -12.60
('offset_PPI_14', 'June26', 'female'): 10.57
('offset_PPI_14', 'June26', 'male'): 21.73
('offset_PPI_14', 'May20', 'female'): 6.41
('offset_PPI_14', 'May20', 'male'): 10.64
('offset_PPI_16', 'April16', 'female'): 28.28
('offset_PPI_16', 'April16', 'male'): 9.34
('offset_PPI_16', 'June26', 'female'): 18.23
('offset_PPI_16',

In [65]:
def average_ppi_by_sex(ppi_dict, label):
    # Collect values by sex
    
    print(f"\nAverage %PPI across dates for {label}:")
    PPF_count = 0
    PPI_count = 0
    PPF = {}
    values_by_sex = {}
    for key, val in ppi_dict.items():
        if isinstance(key, tuple):
            sex = key[-1]  # works for (date, sex) or (experiment, date, sex)
            if pd.notna(val):
                values_by_sex.setdefault(sex, []).append(val)
                if val < 0:
                    PPF_count += 1
                    PPF[key] = val
                else:
                    PPI_count += 1
    print(f"PPI count: {PPI_count}")
    # Average for each sex
    
    avg_by_sex = {sex: np.nanmean(vals) if vals else np.nan for sex, vals in values_by_sex.items()}
    for sex, avg in avg_by_sex.items():
        print(f"{sex}: {avg:.2f}" if pd.notna(avg) else f"{sex}: NaN")
    print(f"mean: {np.mean(list(avg_by_sex.values())):.2f}")
    print(f"PPF count: {PPF_count}")
    for k, v in PPF.items():
        #print(f"{k}: {v:.2f}")
        pass
    print(f"PPF mean: {np.mean(list(PPF.values())):.2f}" if PPF else "PPF mean: NaN")
    return None

# Example usage:

average_ppi_by_sex(ppi_offset, "offset_PPI")
average_ppi_by_sex(ppi_gap, "gap_duration")
average_ppi_by_sex(ppi_ttn, "tone_in_noise")


Average %PPI across dates for offset_PPI:
PPI count: 56
female: 12.77
male: 12.93
mean: 12.85
PPF count: 4
PPF mean: -6.13

Average %PPI across dates for gap_duration:
PPI count: 21
female: 1.89
male: 4.31
mean: 3.10
PPF count: 9
PPF mean: -7.08

Average %PPI across dates for tone_in_noise:
PPI count: 6
female: 30.22
male: 24.18
mean: 27.20
PPF count: 0
PPF mean: NaN


In [66]:
print("\nNumber of negative %PPI cases for gap_duration experiments:")
gap_keys = set(gap_dict.keys())
negative_count = 0
total_count = 0

for key in gap_keys:
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        total_count += 1
        if ppi < 0:
            negative_count += 1

print(f"Negative %PPI cases: {negative_count} out of {total_count}")


Number of negative %PPI cases for gap_duration experiments:
Negative %PPI cases: 9 out of 30


In [67]:
from collections import Counter

print("\nChecking for trends in negative %PPI cases for gap_duration experiments:")

gap_keys = set(gap_dict.keys())
negative_dates = []
negative_sexes = []

for key in gap_keys:
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        if ppi < 0:
            negative_dates.append(date)
            negative_sexes.append(sex)

date_counts = Counter(negative_dates)
sex_counts = Counter(negative_sexes)

print("Negative %PPI by date:", dict(date_counts))
print("Negative %PPI by sex:", dict(sex_counts))


Checking for trends in negative %PPI cases for gap_duration experiments:
Negative %PPI by date: {'April16': 3, 'May20': 5, 'June26': 1}
Negative %PPI by sex: {'male': 3, 'female': 6}


In [68]:
print("\n%PPI for each gap_duration experiment:")
gap_keys = set(gap_dict.keys())

for key in sorted(gap_keys):
    # key is (experiment, date, sex)
    experiment, date, sex = key
    PPS = gap_dict.get(key)
    S = ctrl_dict.get((date, sex), np.nan)
    if pd.notna(S) and pd.notna(PPS) and S != 0:
        ppi = 100 * (S - PPS) / S
        print(f"{key}: %PPI = {ppi:.2f}")
    else:
        print(f"{key}: %PPI = NaN")


%PPI for each gap_duration experiment:
('gap_duration_10', 'April16', 'female'): %PPI = 7.39
('gap_duration_10', 'April16', 'male'): %PPI = 4.01
('gap_duration_10', 'June26', 'female'): %PPI = 1.30
('gap_duration_10', 'June26', 'male'): %PPI = 8.15
('gap_duration_10', 'May20', 'female'): %PPI = -4.72
('gap_duration_10', 'May20', 'male'): %PPI = 12.46
('gap_duration_20', 'April16', 'female'): %PPI = 4.01
('gap_duration_20', 'April16', 'male'): %PPI = 8.33
('gap_duration_20', 'June26', 'female'): %PPI = 4.89
('gap_duration_20', 'June26', 'male'): %PPI = 4.77
('gap_duration_20', 'May20', 'female'): %PPI = -10.54
('gap_duration_20', 'May20', 'male'): %PPI = 20.47
('gap_duration_4', 'April16', 'female'): %PPI = 13.18
('gap_duration_4', 'April16', 'male'): %PPI = -7.50
('gap_duration_4', 'June26', 'female'): %PPI = -0.02
('gap_duration_4', 'June26', 'male'): %PPI = 2.00
('gap_duration_4', 'May20', 'female'): %PPI = -20.93
('gap_duration_4', 'May20', 'male'): %PPI = 2.67
('gap_duration_50', 

In [69]:
def calc_percent_ppi(ctrl_dict, exp_dict, label):
    # If exp_dict keys include experiment, average over experiments for each (date, sex)
    percent_ppi = {}
    # Collect values by (date, sex)
    values_by_date_sex = {}
    for key, PPS in exp_dict.items():
        if isinstance(key, tuple) and len(key) == 2:
            date, sex = key
        elif isinstance(key, tuple) and len(key) == 3:
            _, date, sex = key
        else:
            continue
        S = ctrl_dict.get((date, sex), np.nan)
        if pd.notna(S) and pd.notna(PPS) and S != 0:
            ppi_val = 100 * (S - PPS) / S
        else:
            ppi_val = np.nan
        # Group by (date, sex)
        if (date, sex) not in values_by_date_sex:
            values_by_date_sex[(date, sex)] = []
        values_by_date_sex[(date, sex)].append(ppi_val)
    # Average over experiments for each (date, sex)
    for k, vlist in values_by_date_sex.items():
        vlist = [v for v in vlist if pd.notna(v)]
        percent_ppi[k] = np.nanmean(vlist) if vlist else np.nan
    print(f"\n%PPI for {label} (averaged over experiments if multiple):")
    for k, v in percent_ppi.items():
        print(f"{k}: {v:.2f}" if pd.notna(v) else f"{k}: NaN")
    return percent_ppi

# Calculate %PPI for each experiment type
ppi_ttn = calc_percent_ppi(ctrl_dict, ttn_dict, "tone_in_noise")
ppi_mix = calc_percent_ppi(ctrl_dict, mix_dict, "selected experiments")
ppi_gap = calc_percent_ppi(ctrl_dict, gap_dict, "gap_duration")
ppi_offset = calc_percent_ppi(ctrl_dict, offset_dict, "offset_PPI")


%PPI for tone_in_noise (averaged over experiments if multiple):
('April16', 'female'): 31.86
('April16', 'male'): 18.82
('June26', 'female'): 42.30
('June26', 'male'): 27.26
('May20', 'female'): 16.49
('May20', 'male'): 26.46

%PPI for selected experiments (averaged over experiments if multiple):
('April16', 'female'): 17.49
('April16', 'male'): 7.04
('June26', 'female'): 10.30
('June26', 'male'): 10.16
('May20', 'female'): -0.36
('May20', 'male'): 12.95

%PPI for gap_duration (averaged over experiments if multiple):
('April16', 'female'): 11.77
('April16', 'male'): -0.57
('June26', 'female'): 3.60
('June26', 'male'): 5.34
('May20', 'female'): -9.70
('May20', 'male'): 8.15

%PPI for offset_PPI (averaged over experiments if multiple):
('April16', 'female'): 20.35
('April16', 'male'): 10.85
('June26', 'female'): 13.65
('June26', 'male'): 12.57
('May20', 'female'): 4.31
('May20', 'male'): 15.35


In [70]:
def count_negative_ppi(ppi_dict, label):
    negatives = [v for v in ppi_dict.values() if pd.notna(v) and v < 0]
    print(f"\nNumber of negative %PPI cases for {label}: {len(negatives)}")
    return len(negatives)

# Count negative %PPI cases for each
count_negative_ppi(ppi_ttn, "tone_in_noise")
#count_negative_ppi(ppi_mix, "selected experiments")
count_negative_ppi(ppi_gap, "gap_duration")
count_negative_ppi(ppi_offset, "offset_PPI")


Number of negative %PPI cases for tone_in_noise: 0

Number of negative %PPI cases for gap_duration: 2

Number of negative %PPI cases for offset_PPI: 0


0

In [71]:
print("\nDifferences between %PPI groups per (date, sex):")
group_dicts = [ppi_ttn, ppi_gap, ppi_offset]
group_labels = ["tone_in_noise", "gap_duration", "offset_PPI"]

# Get all unique (date, sex) pairs present in any group
all_keys = set()
for d in group_dicts:
    all_keys.update(d.keys())

for key in sorted(all_keys):
    vals = []
    for d in group_dicts:
        v = d.get(key)
        vals.append(v if pd.notna(v) else None)
    # Only compare if at least two groups have values for this (date, sex)
    for i in range(len(vals)):
        for j in range(i+1, len(vals)):
            v1 = vals[i]
            v2 = vals[j]
            if v1 is not None and v2 is not None:
                # t-test not meaningful for single values, so just compare directly
                # If you have replicates per group, you can use arrays and t-test
                # Here, just check difference and print p-value as NA
                if v1 != v2:
                    mean_diff = v1 - v2
                    print(f"{key}: {group_labels[i]} ({v1:.2f}) vs {group_labels[j]} ({v2:.2f}) | Mean diff: {mean_diff:.2f}")


Differences between %PPI groups per (date, sex):
('April16', 'female'): tone_in_noise (31.86) vs gap_duration (11.77) | Mean diff: 20.09
('April16', 'female'): tone_in_noise (31.86) vs offset_PPI (20.35) | Mean diff: 11.51
('April16', 'female'): gap_duration (11.77) vs offset_PPI (20.35) | Mean diff: -8.58
('April16', 'male'): tone_in_noise (18.82) vs gap_duration (-0.57) | Mean diff: 19.39
('April16', 'male'): tone_in_noise (18.82) vs offset_PPI (10.85) | Mean diff: 7.97
('April16', 'male'): gap_duration (-0.57) vs offset_PPI (10.85) | Mean diff: -11.41
('June26', 'female'): tone_in_noise (42.30) vs gap_duration (3.60) | Mean diff: 38.70
('June26', 'female'): tone_in_noise (42.30) vs offset_PPI (13.65) | Mean diff: 28.65
('June26', 'female'): gap_duration (3.60) vs offset_PPI (13.65) | Mean diff: -10.05
('June26', 'male'): tone_in_noise (27.26) vs gap_duration (5.34) | Mean diff: 21.92
('June26', 'male'): tone_in_noise (27.26) vs offset_PPI (12.57) | Mean diff: 14.69
('June26', 'male